In [2]:
#Load Libraries
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import Dataset
from torch.autograd import Variable
from PIL import Image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
#import helper
import numpy as np


# GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(123)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(123)

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

In [3]:
# Download and load the training data
fashion_trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
fashion_trainloader = torch.utils.data.DataLoader(fashion_trainset, batch_size=128, shuffle=True)

# Download and load the test data
fashion_testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
fashion_testloader = torch.utils.data.DataLoader(fashion_testset, batch_size=128, shuffle=True)

In [4]:
# Download and load the training data
mnist_trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, shuffle=True)

# Download and load the test data
mnist_testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
mnist_testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=128, shuffle=True)

In [147]:
for i, j in fashion_trainloader:
    

TypeError: 'DataLoader' object is not subscriptable

In [4]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [5]:
class SVM:
		# set learning_rate, lambda, n iterations
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

		# SVM fit function
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        y_ = np.where(y <= 0, -1, 1)
        
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

		# SVM predict function
    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #n 28 28 3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(14*14*16, 16)  # Fully connected layer
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 14*14*16)
        x = self.fc1(x)
        return x


In [7]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=7, stride=1, padding=0))
        # Last ImgIn shape=(?, 4, 4, 128)
        #    Conv      ->(?, 4, 4, 16)
        #    Pool      ->(?, 1, 1, 16)
        
        self.last_layer = torch.nn.Sequential(
            torch.nn.Conv2d(128,16,kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=4, stride=1, padding=0)
        )
        
        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))

        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 2, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.last_layer(out)
        #out = out.view(out.size(0), -1)   # Flatten them for FC
        #out = self.layer4(out)
        #out = self.fc2(out)
        return out

In [7]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.drop_prob = 0.5

        # define layer1
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=5, stride=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=1))

        # define layer2
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=5, stride=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=1))

        # define fully connected layer (1024)
        self.fc1 = torch.nn.Linear(18 * 18 * 64, 1024, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer3 = torch.nn.Sequential(
            self.fc1,
            torch.nn.Dropout(p= self.drop_prob))

            
        # define fully connected layer (10 classes)
        self.fc2 = torch.nn.Linear(1024, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    # define feed-forward
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer3(out)
        out = self.fc2(out)
        return out

In [8]:
class multiClassHingeLoss(nn.Module):
    def __init__(self, p=1, margin=1, weight=None, size_average=True):
        super(multiClassHingeLoss, self).__init__()
        self.p=p
        self.margin=margin
        self.weight=weight
        self.size_average=size_average

    # define feed-forward		
    def forward(self, output, y):
        output_y=output[torch.arange(0,y.size()[0]).long(),y.data].view(-1,1)
        print("hinge : ",output_y)
        # output - output(y) + output(i)
        loss=output-output_y+self.margin

        # remove i=y items
        loss[torch.arange(0,y.size()[0]).long(),y.data]=0
        
				# apply max function
        loss[loss<0]=0
        
				# apply power p function
        if(self.p!=1):
            loss=torch.pow(loss,self.p)

        # add weight
        if(self.weight is not None):
            loss=loss*self.weight

        # sum up
        loss=torch.sum(loss)

        if(self.size_average):
            loss/=output.size()[0]

        return loss

In [8]:
#Set hyper-parameter
learning_rate = 0.001
#training_epochs = 50
training_epochs = 3
# 해당 논문에서는 만번의 epoch를 수행했지만 computation power로 인해 epoch 50회 수행
batch_size = 128

In [11]:
# MNIST CNN 모델 정의
mnist_model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(mnist_model.parameters(), lr=learning_rate)

total_batch = len(mnist_trainloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 469


In [12]:
# mnist_model(CNN)
for epoch in range(1):
    avg_cost = 0

    for X, Y in mnist_trainloader:
        X = X.to(device)
        for i in range(len(Y)):
            if Y[i] != '1': Y[i] = 0
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = mnist_model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

In [ ]:
# mnist_model(CNN)
with torch.no_grad():
    correct = 0
    total = 0
    for X_test, Y_test in mnist_testloader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
        prediction = mnist_model(X_test)
        print(prediction)
        predicted = torch.argmax(prediction, 1)
        total += Y_test.size(0)
        correct += (predicted == Y_test).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.28 %


In [222]:
# MNIST CNN+SVM 모델 정의
mnist_SVM_model = CNN().to(device)

criterion = multiClassHingeLoss().to(device)
optimizer = torch.optim.Adam(mnist_SVM_model.parameters(), lr=learning_rate)

total_batch = len(mnist_trainloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 469


In [223]:
# minst_SVM_model(CNN + SVM)
for epoch in range(1):
    avg_cost = 0

    for X, Y in mnist_trainloader: 
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = mnist_SVM_model(X)
        cost = criterion(hypothesis, Y)
        print(cost)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


tensor(15785.0293, grad_fn=<DivBackward0>)
tensor(12078.8193, grad_fn=<DivBackward0>)
tensor(8815.7100, grad_fn=<DivBackward0>)
tensor(6404.5293, grad_fn=<DivBackward0>)
tensor(5511.2612, grad_fn=<DivBackward0>)
tensor(2911.8611, grad_fn=<DivBackward0>)


KeyboardInterrupt: 

In [ ]:
# mnist_SVM_model(CNN+SVM)
with torch.no_grad():
    correct = 0
    total = 0
    for X_test, Y_test in mnist_testloader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
        prediction = mnist_SVM_model(X_test)
        predicted = torch.argmax(prediction, 1)
        total += Y_test.size(0)
        correct += (predicted == Y_test).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.25 %


In [6]:
# fashion-MNIST CNN 모델 정의
fashion_model = CNN().to(device)

criterion = torch.nn.MSELoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(fashion_model.parameters(), lr=learning_rate)

total_batch = len(fashion_trainloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 469


In [11]:
print(len(fashion_trainloader))
X_trains = []
X_trains_correct = []

469


In [12]:
# fashion_model(CNN)
for epoch in range(2):
    avg_cost = 0
    for X, Y in (fashion_trainloader):
        X = X.to(device)
        for i in range(len(Y)):
            if Y[i] == 0: Y[i] = 1
            else: Y[i] = 0
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = fashion_model(X)
        #128개의 배치 이미지 4*4 128개
        if epoch == 1:
            for hypo in range(len(hypothesis)):
                x = hypothesis[hypo].tolist()
                y = Y[hypo].item()
            #print(y)
                X_trains.append(x)
                X_trains_correct.append(y)

        cost = criterion(hypothesis.to(torch.float32), Y.to(torch.float32))

        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch

    print(hypothesis.shape)
    print(hypothesis[0])
    print(cost.shape)
    print(cost.item())
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


c:\Users\asuin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([96])) that is different to the input size (torch.Size([96, 128, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([96, 128, 1, 1])
tensor([[[0.0953]],

        [[0.0967]],

        [[0.0968]],

        [[0.0925]],

        [[0.0979]],

        [[0.0948]],

        [[0.0967]],

        [[0.0948]],

        [[0.0938]],

        [[0.0946]],

        [[0.0983]],

        [[0.0997]],

        [[0.0981]],

        [[0.0949]],

        [[0.0929]],

        [[0.0971]],

        [[0.0940]],

        [[0.0895]],

        [[0.0970]],

        [[0.0925]],

        [[0.0977]],

        [[0.0939]],

        [[0.0986]],

        [[0.0956]],

        [[0.0994]],

        [[0.0955]],

        [[0.0966]],

        [[0.0951]],

        [[0.0969]],

        [[0.0963]],

        [[0.0992]],

        [[0.0942]],

        [[0.0965]],

        [[0.0971]],

        [[0.0976]],

        [[0.0982]],

        [[0.0972]],

        [[0.0999]],

        [[0.0945]],

        [[0.0922]],

        [[0.0991]],

        [[0.0946]],

        [[0.0951]],

        [[0.0975]],

        [[0.0931]],

        [[0.0963]],

      

In [213]:
print(len(X_trains))

60000


In [216]:
print(len(X_trains))
print(len(X_trains[0]))

60000
128


NameError: name 'pritn' is not defined

In [157]:
print(len(X_trains))
print(X_trains[0])

60000
[0.0356176495552063, 0.29730224609375, 0.09643476456403732, 0.05676256865262985, 0.13543550670146942, 0.013553299009799957, -0.36655616760253906, -0.12852045893669128, 0.1983237862586975, -0.11612866818904877, 0.23711320757865906, -0.22383151948451996, -0.11666763573884964, -0.0069043636322021484, 0.04707544296979904, -0.2760394811630249]


In [200]:
from sklearn.svm import OneClassSVM
clf = OneClassSVM(gamma=0.001, nu=0.01, kernel='rbf')
clf.fit(X_trains)

OneClassSVM(gamma=0.001, nu=0.01)

In [201]:
x_predict = clf.predict(X_trains)

In [202]:
correct = 0
for i in range(60000):
    if x_predict[i] == X_trains_correct[i]: correct+=1
print(correct)
print('acuracy: ', correct*100//60000)

5945
acuracy:  9


In [166]:
X_tests = []

In [27]:
# fashion_model(CNN)
with torch.no_grad():
    correct = 0
    total = 0
    for X_test, Y_test in fashion_testloader:
        X_test = X_test.to(device)
        for i in range(len(Y_test)):
            if Y_test[i] == 0: Y_test[i] = 1
            else: Y_test[i] = 0
        Y_test = Y_test.to(device)
        prediction = fashion_model(X_test)

        for idx in range(len(prediction)):
            X_tests.append([[prediction[idx][0].item()], [prediction[idx][1].item()],[Y_test[idx].item()]])
        predicted = torch.argmax(prediction, 1)
        total += Y_test.size(0)
        correct += (predicted == Y_test).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


RuntimeError: mat1 and mat2 shapes cannot be multiplied (65536x4 and 2048x625)

In [168]:
print(len(X_tests))

10000


In [170]:
for i in X_tests:
    print(clf.predict(i))

[ 1  1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[-1  1 -1]
[-1 -1 -1]
[-1 -1  1]
[ 1 -1 -1]
[ 1 -1 -1]
[ 1  1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1  1 -1]
[-1 -1  1]
[ 1  1 -1]
[-1 -1  1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1  1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1  1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1  1 -1]
[-1  1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1  1]
[-1 -1 -1]
[-1  1 -1]
[-1  1 -1]
[ 1 -1 -1]
[-1  1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1  1]
[-1 -1 -1]
[-1 -1  1]
[-1 -1 -1]
[-1 -1  1]
[-1  1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]
[ 1 -1 -1]
[-1 -1 -1]

In [19]:
# fashion-MNIST CNN + SVM 모델 정의
fashion_SVM_model = CNN().to(device)

criterion = multiClassHingeLoss().to(device)
optimizer = torch.optim.Adam(fashion_SVM_model.parameters(), lr=learning_rate)

total_batch = len(fashion_trainloader)
print('총 배치의 수 : {}'.format(total_batch))


총 배치의 수 : 469


In [ ]:
# fashion_SVM_model(CNN + SVM)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in fashion_trainloader: 
        X = X.to(device)
        for i in range(len(Y)):
            if Y[i] != '0': Y[i] = 1
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = fashion_SVM_model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

In [30]:
# fashion_SVM_model(CNN + SVM)
with torch.no_grad():
    correct = 0
    total = 0
    for X_test, Y_test in fashion_testloader:
        X_test = X_test.to(device)
        for i in range(len(Y_test)):
            if Y_test[i] != "0": Y_test[i] = 1
        Y_test = Y_test.to(device)

        prediction = fashion_SVM_model(X_test)
        predicted = torch.argmax(prediction, 1)

        total += Y_test.size(0)
        correct += (predicted == Y_test).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 100.0 %


In [41]:
k = 0
with torch.no_grad():
    for X_test, Y_test in fashion_testloader:
        k+=1
        X_test = X_test.to(device)
        for i in range(len(Y_test)):
            if Y_test[i] != "0": Y_test[i] = 1
        Y_test = Y_test.to(device)

        prediction = fashion_SVM_model(X_test)
        predicted = torch.argmax(prediction, 1)

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


tensor(4)
tensor(8)
tensor(5)
tensor(5)
tensor(3)
tensor(7)
tensor(0)
tensor(8)
tensor(7)
tensor(2)
tensor(4)
tensor(7)
tensor(6)
tensor(4)
tensor(2)
tensor(4)
tensor(4)
tensor(8)
tensor(3)
tensor(9)
tensor(2)
tensor(1)
tensor(8)
tensor(5)
tensor(8)
tensor(4)
tensor(2)
tensor(6)
tensor(7)
tensor(2)
tensor(0)
tensor(4)
tensor(4)
tensor(4)
tensor(9)
tensor(2)
tensor(0)
tensor(2)
tensor(7)
tensor(2)
tensor(3)
tensor(1)
tensor(7)
tensor(9)
tensor(8)
tensor(4)
tensor(6)
tensor(4)
tensor(3)
tensor(9)
tensor(5)
tensor(2)
tensor(9)
tensor(5)
tensor(4)
tensor(8)
tensor(5)
tensor(7)
tensor(5)
tensor(2)
tensor(2)
tensor(1)
tensor(5)
tensor(1)
tensor(7)
tensor(9)
tensor(5)
tensor(2)
tensor(7)
tensor(5)
tensor(0)
tensor(9)
tensor(8)
tensor(5)
tensor(6)
tensor(9)
tensor(3)
tensor(0)
tensor(5)
tensor(4)
tensor(5)
tensor(5)
tensor(8)
tensor(0)
tensor(7)
tensor(6)
tensor(2)
tensor(1)
tensor(4)
tensor(6)
tensor(4)
tensor(7)
tensor(7)
tensor(3)
tensor(9)
tensor(7)
tensor(2)
tensor(4)
tensor(8)
tensor(3)
